In [1]:

import torch
import math
from utils.aihub_kpop_dataset import KpopImageDatasetwT
from utils.eval_posehrnet import HeatmapGEval
from torch.utils.data import DataLoader
from model import PoseHRNet
from torch import nn, Tensor

import tqdm
from tqdm import tqdm_notebook





device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device ? : {}'.format(device))
mymodel = PoseHRNet('w32', num_joints=29)
mydataset = KpopImageDatasetwT()
mydataloader = DataLoader(dataset=mydataset, batch_size=1, shuffle=True)
myloss = nn.MSELoss()
myoptimizer = torch.optim.Adam(mymodel.parameters(), lr=1e-3)
myeval = HeatmapGEval()


device ? : cpu


In [2]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 1 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [3]:
def eval(dataloader, model, eval):
    size = len(dataloader.dataset)
    model.eval()
    
    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            pred = model(X)
            eval_result = eval(pred, y)
            print(eval_result)
            break
                

In [4]:
epochs = 1
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    # train(mydataloader, mymodel, myloss, myoptimizer)
    eval(mydataloader, mymodel, myeval)
    
    
print("Done!")

Epoch 1
-------------------------------


KeyboardInterrupt: 